<a target="_blank" href="https://colab.research.google.com/github/sekijima-lab/DiffInt/blob/main/colab/DiffInt_generate.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#@title you need to check the numpy version = 1.23.5. Don't restart session
!pip uninstall -y numpy
!pip install numpy==1.23.5

#import numpy
#print("NumPy version:", numpy.__version__)

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Install dependencies (about 5 minutes)
%cd /content

import os

commands = [
    "conda install openbabel -c conda-forge",
    "conda install conda-forge/label/cf202003::oddt",
    "pip install scipy==1.9.3 scikit-learn==1.1.3 --no-cache-dir --force-reinstall",
    "pip install torch==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu118",
    "pip install torch torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118",
    "pip install pytorch-lightning==1.8.4",
    "pip install rdkit==2022.9.2",
    "pip install biopython==1.79",
    "pip install imageio==2.21.2",
    "pip install pyg-lib torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html",
    "pip install networkx==2.8.6",
    "pip install py3Dmol",
    "pip install wandb==0.13.1",
    "pip install seaborn",
]

errors = {}

if not os.path.isfile("/content/READY"):
  for cmd in commands:
    # os.system(cmd)
    with os.popen(cmd) as f:
      out = f.read()
      status = f.close()

    if status is not None:
      errors[cmd] = out
      print(f"\n\nAn error occurred while running '{cmd}'\n")
      print("Status:\t", status)
      print("Message:\t", out)

if len(errors) == 0:
  os.system("touch /content/READY")

In [ ]:
%env MPLBACKEND=Agg

In [ ]:
!git clone https://github.com/sekijima-lab/DiffInt.git

In [ ]:
#@title Change Directory
import os
os.chdir('DiffInt')

In [ ]:
#@title Data construction of 10 molecules
!python test_single.py --sdf example/1a2g_A_rec.sdf --pdb example/1a2g_A_rec.pdb --checkpoint checkpoints/best_model.ckpt --n_samples 10 --outdir tmp_out

In [ ]:
from rdkit import Chem
import py3Dmol

In [ ]:
#@title Ooen PDB for Visualization
with open("example/1a2g_A_rec.pdb") as f:
    system = "".join([x for x in f])

In [ ]:
#@title Read Generated Molecules
suppl = Chem.SDMolSupplier('tmp_out/processed/1a2g_A_rec_gen.sdf')
out_sdf = [mol for mol in suppl if mol is not None]
print(f"number of molecules: {len(out_sdf)}")

In [ ]:
#@title Visualization
viewA = py3Dmol.view(width=400, height=300)
id = 0  # select number of molecules (0-9)
mol_block = Chem.MolToMolBlock(out_sdf[id])
viewA.addModel(mol_block, 'sdf')
viewA.setStyle({'stick': {}})
viewA.addModelsAsFrames(system)
viewA.addSurface(py3Dmol.VDW,{'opacity':0.9,'color':'white'})

viewA.zoomTo()
viewA.show()